## Face-tracking
This script can track and identify multiple faces and store last 5 centroids of bboxe predictions and also visualizes them

In [1]:
import glob
import cv2
import dlib
import face_recognition
import numpy as np
import os
import glob

def load_encoding_images(images_path):
        # Load Images
        images_path = glob.glob(os.path.join(images_path, "*.*"))

        print("{} encoding images found.".format(len(images_path)))

        for img_path in images_path:
            img = cv2.imread(img_path)
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            basename = os.path.basename(img_path)
            (filename, ext) = os.path.splitext(basename)
            # Get encoding
            img_encoding = face_recognition.face_encodings(rgb_img)[0]
            
            known_face_encodings.append(img_encoding)
            known_face_names.append(filename)
        print("Encoding images loaded")

In [2]:

trackers = []
face_names = []
known_face_encodings = []  # Pre-loaded known face encodings
known_face_names = []  # Corresponding names of known faces
load_encoding_images('encoding')
ground_truth_values = [(659, 383, 586, 694), (615, 371, 561, 706), (499, 306, 534, 725), (549, 344, 547, 732), (669, 358, 573, 720),
     (664, 382, 580, 681), (487, 291, 526, 752), (469, 265, 523, 744), (667, 378, 576, 700), (659, 344, 570, 720),
     (647, 382, 580, 701), (480, 279, 526, 750), (673, 377, 577, 700), (659, 344, 570, 720), (636, 379, 571, 698),
     (673, 367, 573, 703), (522, 321, 538, 749), (670, 379, 577, 700), (471, 267, 528, 745), (583, 356, 555, 720)]
frame_index = 0 

1 encoding images found.
Encoding images loaded


In [6]:
import cv2
import dlib
import face_recognition
import numpy as np

# for webcam
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture("sampl-vedio.mp4")
face_detector = dlib.get_frontal_face_detector()

# Initialize lists for trackers and face names
trackers = []
face_names = []

# Initialize a list to store centroids of the last 5 frames
centroids_history = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to RGB (dlib uses RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces in the frame
    detections = face_detector(rgb_frame)

    # Clear previous trackers
    trackers = []

    # Update centroids history with current detections
    current_centroids = []
    current_names = []
    for detection in detections:
        x1, y1, x2, y2 = int(detection.left()), int(detection.top()), int(detection.right()), int(detection.bottom())

        # Calculate the centroid
        centroid_x = int((x1 + x2) / 2)
        centroid_y = int((y1 + y2) / 2)

        # Store current centroid in history
        current_centroids.append((centroid_x, centroid_y))

        # Face Recognition (you can modify this part as per your recognition logic)
        face_encodings = face_recognition.face_encodings(rgb_frame, [(y1, x2, y2, x1)])

        if face_encodings:
            face_encoding = face_encodings[0]
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
            else:
                known_face_encodings.append(face_encoding)
                name = f"Person {len(known_face_names) + 1}"
                known_face_names.append(name)

            current_names.append(name)

    # Update centroids history with current centroids
    centroids_history.append((current_centroids, current_names))

    # Keep only the last 5 frames' centroids in history
    if len(centroids_history) > 5:
        centroids_history = centroids_history[-5:]

    # Display centroids from previous 5 frames (without bounding boxes and names)
    for prev_centroids, _ in centroids_history[:-1]:  # exclude the current frame
        for (centroid_x, centroid_y) in prev_centroids:
            # Draw circle at centroid for past frames
            cv2.circle(frame, (centroid_x, centroid_y), 5, (255, 255, 0), -1)

    # Display bounding box and name for current frame's centroids
    for (centroid_x, centroid_y), name in zip(current_centroids, current_names):
        # Calculate the bounding box coordinates based on face dimensions
        face_width = abs(x2 - x1)
        face_height = abs(y2 - y1)
        bbox_x1 = centroid_x - face_width // 2
        bbox_y1 = centroid_y - face_height // 2
        bbox_x2 = centroid_x + face_width // 2
        bbox_y2 = centroid_y + face_height // 2

        # Draw bounding box around current centroid
        cv2.rectangle(frame, (bbox_x1, bbox_y1), (bbox_x2, bbox_y2), (0, 255, 0), 2)

        # Display dynamically fetched name next to current centroid
        cv2.putText(frame, f"My Name: {name}", (bbox_x1, bbox_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Video', frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()


## This script works same as prev but doesnt store or display prev centroids

In [9]:
import cv2
import dlib
import face_recognition
import numpy as np

# Initialize video capture from file
cap = cv2.VideoCapture(0)

# Initialize known face encodings and names
known_face_encodings = []
known_face_names = []

# Load encoding images and initialize face detector
load_encoding_images("encoding")
face_detector = dlib.get_frontal_face_detector()

# Initialize lists for centroids and face names
centroids = []
face_names = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to RGB (dlib uses RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces in the frame
    detections = face_detector(rgb_frame)

    # Iterate over detected faces
    for detection in detections:
        x1, y1, x2, y2 = int(detection.left()), int(detection.top()), int(detection.right()), int(detection.bottom())

        # Calculate the centroid
        centroid_x = int((x1 + x2) / 2)
        centroid_y = int((y1 + y2) / 2)

        # Face Recognition
        face_encodings = face_recognition.face_encodings(rgb_frame, [(y1, x2, y2, x1)])

        if face_encodings:
            face_encoding = face_encodings[0]
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
            else:
                known_face_encodings.append(face_encoding)
                name = f"Person {len(known_face_names) + 1}"
                known_face_names.append(name)

            # Draw bounding box around face
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Track centroid with circle
            cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 0, 255), -1)
            cv2.putText(frame, name, (centroid_x - 20, centroid_y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Display the frame
    cv2.imshow('Video', frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()


1 encoding images found.
Encoding images loaded
